In [7]:
%run func.ipynb
import mahotas
from pyfeats import *
import pandas as pd
import numpy as np
import time

from sklearn.feature_selection import f_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv('train.csv')
df.fillna(df.mode().iloc[0], inplace=True) # Replace NaN values with mode of corresponding column
# use the factorize() function to convert the column of strings to integers
non_int_columns = df.select_dtypes(exclude=['int']).columns.tolist()
for i in non_int_columns:
    df[i] = pd.factorize(df[i])[0] + 1

In [3]:
df.head()

Id  MSSubClass  MSZoning  LotFrontage  LotArea  Street  Alley  LotShape  \
0   1          60         1            1     8450       1      1         1   
1   2          20         1            2     9600       1      1         1   
2   3          60         1            3    11250       1      1         2   
3   4          70         1            4     9550       1      1         2   
4   5          60         1            5    14260       1      1         2   

   LandContour  Utilities  ...  PoolArea  PoolQC  Fence  MiscFeature  MiscVal  \
0            1          1  ...         0       1      1            1        0   
1            1          1  ...         0       1      1            1        0   
2            1          1  ...         0       1      1            1        0   
3            1          1  ...         0       1      1            1        0   
4            1          1  ...         0       1      1            1        0   

   MoSold  YrSold  SaleType  SaleCondition  SalePrice  
0       2    2008         1              1     208500  
1       5    2007         1              1     181500  
2       9    2008         1              1     223500  
3       2    2006         1              2     140000  
4      12    2008         1              1     250000  

[5 rows x 81 columns]

In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

len_row, len_col = X.shape
print('Row: ',len_row)

X_col = X.columns.tolist()

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the dataset
scaler.fit(X)

# Transform the dataset using the scaler
X_std = scaler.transform(X)

X_std = pd.DataFrame(data=X_std, columns=X_col)

Row:  1460


In [5]:
X_std.head()

Id  MSSubClass  MSZoning  LotFrontage   LotArea    Street     Alley  \
0 -1.730865    0.073375 -0.426561    -0.896540 -0.207142 -0.064238 -0.169981   
1 -1.728492   -0.872563 -0.426561    -0.859423 -0.091886 -0.064238 -0.169981   
2 -1.726120    0.073375 -0.426561    -0.822305  0.073480 -0.064238 -0.169981   
3 -1.723747    0.309859 -0.426561    -0.785188 -0.096897 -0.064238 -0.169981   
4 -1.721374    0.073375 -0.426561    -0.748070  0.375148 -0.064238 -0.169981   

   LotShape  LandContour  Utilities  ...  ScreenPorch  PoolArea    PoolQC  \
0 -0.701291    -0.304158   -0.02618  ...    -0.270208 -0.068692 -0.049718   
1 -0.701291    -0.304158   -0.02618  ...    -0.270208 -0.068692 -0.049718   
2  1.016637    -0.304158   -0.02618  ...    -0.270208 -0.068692 -0.049718   
3  1.016637    -0.304158   -0.02618  ...    -0.270208 -0.068692 -0.049718   
4  1.016637    -0.304158   -0.02618  ...    -0.270208 -0.068692 -0.049718   

      Fence  MiscFeature   MiscVal    MoSold    YrSold  SaleType  \
0 -0.282681    -0.054116 -0.087688 -1.599111  0.138777 -0.292103   
1 -0.282681    -0.054116 -0.087688 -0.489110 -0.614439 -0.292103   
2 -0.282681    -0.054116 -0.087688  0.990891  0.138777 -0.292103   
3 -0.282681    -0.054116 -0.087688 -1.599111 -1.367655 -0.292103   
4 -0.282681    -0.054116 -0.087688  2.100892  0.138777 -0.292103   

   SaleCondition  
0      -0.394337  
1      -0.394337  
2      -0.394337  
3       0.732340  
4      -0.394337  

[5 rows x 80 columns]

In [13]:
X_selected.head()

OverallQual  GrLivArea  GarageCars  TotalBsmtSF  GarageArea  1stFlrSF  \
1090    -2.241782  -0.905127    0.311725    -2.411167   -0.341457 -0.317311   
468      1.374795   0.225654    1.650307     1.296463    1.838848  1.219736   
1007    -1.518467  -0.806136   -1.026858    -1.166169   -0.874836 -1.595596   
1194    -0.071836  -0.311181   -1.026858    -0.899384   -0.814012 -1.292845   
859     -0.071836   2.167400    0.311725    -0.007820    0.683193  0.904046   

      FullBath  YearRemodAdd  TotRmsAbvGrd  YearBuilt  ...   MiscVal  \
1090  0.789741     -1.689368     -0.318683  -0.704406  ... -0.087688   
468   0.789741      1.024029      0.296763   1.150356  ... -0.087688   
1007 -1.026041     -0.720298     -0.318683  -0.041991  ... -0.087688   
1194 -1.026041     -0.768752      0.296763  -0.075111  ... -0.087688   
859   0.789741     -0.041949      1.527656  -0.108232  ... -0.087688   

         Alley  LowQualFinSF  Utilities  BsmtHalfBath  MiscFeature  \
1090 -0.169981     -0.120242   -0.02618     -0.241061    -0.054116   
468  -0.169981     -0.120242   -0.02618     -0.241061    -0.054116   
1007 -0.169981     -0.120242   -0.02618     -0.241061    -0.054116   
1194 -0.169981     -0.120242   -0.02618     -0.241061    -0.054116   
859  -0.169981     -0.120242   -0.02618     -0.241061    -0.054116   

      MasVnrType  BsmtFinSF2  Condition2  BsmtQual  
1090    0.312806   -0.288653   -0.091553  0.406942  
468     1.904082   -0.288653   -0.091553 -0.984475  
1007    0.312806    1.404223   -0.091553  0.406942  
1194    1.904082   -0.288653   -0.091553  0.406942  
859    -1.278471   -0.288653   -0.091553  0.406942  

[5 rows x 80 columns]

In [36]:
# # without feature selection
# X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.2, random_state = 93)
# start_time = time.time()
# # Train a random forest model on the training data
# model = GradientBoostingRegressor(random_state = 10)
# model.fit(X_train, y_train)
# end_time = time.time()

# # Evaluate the model on the testing data
# y_pred = model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)

# print(f'Mean Squared Error = {mse}')
# print(f"Elapsed time: {end_time - start_time} seconds")
# print()

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.2, random_state = 93)

# Define the range of values for k
k_values = range(1, len_col+1)
# Initialize dictionaries to store results
mses = {}
selected_each_k = {}
timer = {}

for k in k_values:
    scores, selected, scores_ith, score_df, relevancy, redundancy = min_redun_max_relev(X, y, k)
    selected_each_k[f'k = {k}'] = selected
    # Remove all columns from X that are not selected
    X_selected = X_train[selected]
    
    start_time = time.time()
    # Train a random forest model on the training data
    model = GradientBoostingRegressor(random_state = 10)
    model.fit(X_selected, y_train)
    end_time = time.time()

    # Evaluate the model on the testing data
    X_selected_test = X_test[selected]
    y_pred = model.predict(X_selected_test)
    mse = mean_squared_error(y_test, y_pred)
    
    times = end_time - start_time
    
    timer[f'k = {k}'] = times
    mses[f'k = {k}'] = mse

In [22]:
# Choose the best k based on the validation accuracies

print(f"MSE using all columns= {mses[f'k = {len_col}']}")
print(f"Elapsed Time = {timer[f'k = {len_col}']}")
print()
best_k = min(mses, key=mses.get)
print(f"Best {best_k} with MSE = {mses[best_k]}")
print(f"Elapsed Time = {timer[best_k]}")
print()
print(f"MSE using 3 best columns= {mses[f'k = 3']}")
print(f"Elapsed Time = {timer[f'k = 3']}")
print()
print(f"MSE using 10 best columns= {mses[f'k = 10']}")
print(f"Elapsed Time = {timer[f'k = 10']}")
print()

MSE using all columns= 412026380.4704741
Elapsed Time = 0.6538717746734619

Best k = 67 with MSE = 372592607.604737
Elapsed Time = 0.5721695423126221

MSE using 3 best columns= 895863183.6754111
Elapsed Time = 0.1114511489868164

MSE using 10 best columns= 587507015.6618106
Elapsed Time = 0.2785656452178955



In [46]:
worsts = [['BsmtFinSF2','Condition2','BsmtQual'],['MiscVal','Alley','LowQualFinSF','Utilities','BsmtHalfBath','MiscFeature','MasVnrType','BsmtFinSF2','Condition2','BsmtQual']]

for worst in worsts:
    # USING 3 Worst columns & USING 10 Worst columns
    X_select = X_selected[worst]

    start_time = time.time()
    # Train a random forest model on the training data
    model = GradientBoostingRegressor()
    model.fit(X_select, y_train)
    end_time = time.time()

    # Evaluate the model on the testing data
    X_selected_test = X_test[worst]
    y_pred = model.predict(X_selected_test)
    mse = mean_squared_error(y_test, y_pred)

    times = end_time - start_time

    print(f"MSE using {len(worst)} worst columns= {mse}")
    print(f"Elapsed Time = {times}")
    print()

MSE using 3 worst columns= 3234911273.926044
Elapsed Time = 0.06067848205566406

MSE using 10 worst columns= 2725846895.670977
Elapsed Time = 0.0585019588470459



In [47]:
print(selected_each_k[best_k])

['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'GarageArea', '1stFlrSF', 'FullBath', 'YearRemodAdd', 'TotRmsAbvGrd', 'YearBuilt', 'Fireplaces', 'BsmtFinSF1', 'HeatingQC', 'Foundation', 'WoodDeckSF', 'BsmtFinType1', 'OpenPorchSF', 'LotArea', '2ndFlrSF', 'MasVnrArea', 'HalfBath', 'LotShape', 'BsmtExposure', 'CentralAir', 'LotFrontage', 'ExterQual', 'RoofStyle', 'BsmtFullBath', 'Electrical', 'GarageFinish', 'PavedDrive', 'HouseStyle', 'BsmtUnfSF', 'FireplaceQu', 'SaleCondition', 'Neighborhood', 'ScreenPorch', 'BedroomAbvGr', 'LotConfig', 'Exterior2nd', 'ExterCond', 'KitchenAbvGr', 'Functional', 'GarageCond', 'PoolQC', 'EnclosedPorch', 'LandContour', 'MSZoning', 'SaleType', 'GarageYrBlt', 'Heating', 'BldgType', 'Exterior1st', 'GarageType', 'PoolArea', 'KitchenQual', 'BsmtCond', 'Fence', '3SsnPorch', 'MoSold', 'MSSubClass', 'OverallCond', 'Condition1', 'Street', 'GarageQual', 'LandSlope', 'BsmtFinType2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']


In [24]:
# Find the maximum number of values across all keys
max_values = max([len(val) if isinstance(val, list) else 1 for val in selected_each_k.values()])

# Iterate through the keys and add "n/a" values as necessary
for key, val in selected_each_k.items():
    # Check if the value is a string and split by comma if necessary
    if isinstance(val, str):
        values = val.split(',')
    else:
        values = val
        
    # If the number of values is less than the maximum, add "n/a" values
    num_values = len(values)
    if num_values < max_values:
        diff = max_values - num_values
        values += ['-'] * diff
    
    # Join the values with commas and update the dictionary
    selected_each_k[key] = values
    
selected_each_k_df = pd.DataFrame(selected_each_k)

In [25]:
selected_each_k_df.iloc[:, :100].head(100)

k = 1        k = 2        k = 3        k = 4        k = 5  \
0   OverallQual  OverallQual  OverallQual  OverallQual  OverallQual   
1             -    GrLivArea    GrLivArea    GrLivArea    GrLivArea   
2             -            -   GarageCars   GarageCars   GarageCars   
3             -            -            -  TotalBsmtSF  TotalBsmtSF   
4             -            -            -            -   GarageArea   
..          ...          ...          ...          ...          ...   
75            -            -            -            -            -   
76            -            -            -            -            -   
77            -            -            -            -            -   
78            -            -            -            -            -   
79            -            -            -            -            -   

          k = 6        k = 7        k = 8        k = 9       k = 10  ...  \
0   OverallQual  OverallQual  OverallQual  OverallQual  OverallQual  ...   
1     GrLivArea    GrLivArea    GrLivArea    GrLivArea    GrLivArea  ...   
2    GarageCars   GarageCars   GarageCars   GarageCars   GarageCars  ...   
3   TotalBsmtSF  TotalBsmtSF  TotalBsmtSF  TotalBsmtSF  TotalBsmtSF  ...   
4    GarageArea   GarageArea   GarageArea   GarageArea   GarageArea  ...   
..          ...          ...          ...          ...          ...  ...   
75            -            -            -            -            -  ...   
76            -            -            -            -            -  ...   
77            -            -            -            -            -  ...   
78            -            -            -            -            -  ...   
79            -            -            -            -            -  ...   

         k = 71       k = 72       k = 73       k = 74       k = 75  \
0   OverallQual  OverallQual  OverallQual  OverallQual  OverallQual   
1     GrLivArea    GrLivArea    GrLivArea    GrLivArea    GrLivArea   
2    GarageCars   GarageCars   GarageCars   GarageCars   GarageCars   
3   TotalBsmtSF  TotalBsmtSF  TotalBsmtSF  TotalBsmtSF  TotalBsmtSF   
4    GarageArea   GarageArea   GarageArea   GarageArea   GarageArea   
..          ...          ...          ...          ...          ...   
75            -            -            -            -            -   
76            -            -            -            -            -   
77            -            -            -            -            -   
78            -            -            -            -            -   
79            -            -            -            -            -   

         k = 76       k = 77       k = 78       k = 79       k = 80  
0   OverallQual  OverallQual  OverallQual  OverallQual  OverallQual  
1     GrLivArea    GrLivArea    GrLivArea    GrLivArea    GrLivArea  
2    GarageCars   GarageCars   GarageCars   GarageCars   GarageCars  
3   TotalBsmtSF  TotalBsmtSF  TotalBsmtSF  TotalBsmtSF  TotalBsmtSF  
4    GarageArea   GarageArea   GarageArea   GarageArea   GarageArea  
..          ...          ...          ...          ...          ...  
75  MiscFeature  MiscFeature  MiscFeature  MiscFeature  MiscFeature  
76            -   MasVnrType   MasVnrType   MasVnrType   MasVnrType  
77            -            -   BsmtFinSF2   BsmtFinSF2   BsmtFinSF2  
78            -            -            -   Condition2   Condition2  
79            -            -            -            -     BsmtQual  

[80 rows x 80 columns]

In [26]:
for i in range(len(scores_ith)):
    combines = {'Relevancy':relevancy[i], 'Redundancy':redundancy[i],'MRMR':scores_ith[i]}
    scored_ith = pd.DataFrame(combines)
    scored_ith = scored_ith.sort_values('MRMR', ascending=False)
    print('ITERASI KE ', i+1)
    print(scored_ith)
    print()
    print()

ITERASI KE  1
               Relevancy  Redundancy          MRMR
OverallQual  2436.770591     0.00001  2.436771e+08
GrLivArea    1470.585010     0.00001  1.470585e+08
GarageCars   1013.705666     0.00001  1.013706e+08
GarageArea    926.951287     0.00001  9.269513e+07
TotalBsmtSF   880.341282     0.00001  8.803413e+07
...                  ...         ...           ...
MiscFeature     0.216122     0.00001  2.161222e+04
BsmtFinSF2      0.188780     0.00001  1.887795e+04
MasVnrType      0.166779     0.00001  1.667793e+04
Condition2      0.034062     0.00001  3.406202e+03
BsmtQual        0.023949     0.00001  2.394869e+03

[80 rows x 3 columns]


ITERASI KE  2
               Relevancy  Redundancy         MRMR
GrLivArea    1470.585010    0.593007  2479.876196
1stFlrSF      845.524488    0.476224  1775.477068
GarageCars   1013.705666    0.600671  1687.622916
GarageArea    926.951287    0.562022  1649.315665
TotalBsmtSF   880.341282    0.537808  1636.904743
...                  ...         ..

BsmtQual        0.023949    0.105170    0.227715


ITERASI KE  38
               Relevancy  Redundancy        MRMR
BedroomAbvGr   42.456412    0.118248  359.044752
LotConfig      17.565263    0.054464  322.511999
Exterior2nd    40.660476    0.131134  310.067039
KitchenAbvGr   27.437234    0.092687  296.018848
ExterCond      21.921284    0.075413  290.684809
Functional     17.325200    0.061534  281.553544
GarageCond     18.485024    0.070760  261.236869
PoolQC          9.140489    0.035353  258.546540
EnclosedPorch  24.509275    0.094907  258.246123
BldgType       18.726622    0.077185  242.619019
MSZoning       19.902726    0.082960  239.906899
LandContour    12.448384    0.054825  227.055109
PoolArea       12.556221    0.055897  224.631117
GarageYrBlt    19.641897    0.090302  217.513379
SaleType        7.788912    0.035885  217.052342
Heating        16.781745    0.081275  206.481539
Exterior1st    21.513454    0.105342  204.224197
GarageType     15.263524    0.086553  176.348167
Bsm

ITERASI KE  69
              Relevancy  Redundancy       MRMR
Id             0.700676    0.021247  32.977472
RoofMatl       1.873169    0.059527  31.467439
MiscVal        0.654934    0.023443  27.937807
Alley          1.115961    0.047206  23.640184
LowQualFinSF   0.956600    0.046822  20.430689
Utilities      0.298804    0.021270  14.048448
BsmtHalfBath   0.413789    0.037595  11.006620
MiscFeature    0.216122    0.037755   5.724318
MasVnrType     0.166779    0.044978   3.708059
BsmtFinSF2     0.188780    0.064809   2.912843
Condition2     0.034062    0.035611   0.956501
BsmtQual       0.023949    0.089377   0.267952


ITERASI KE  70
              Relevancy  Redundancy       MRMR
RoofMatl       1.873169    0.058858  31.825038
MiscVal        0.654934    0.023193  28.238078
Alley          1.115961    0.046546  23.975454
LowQualFinSF   0.956600    0.046784  20.447092
Utilities      0.298804    0.021154  14.124919
BsmtHalfBath   0.413789    0.037342  11.081119
MiscFeature    0.216122    0